In [ ]:
%run main.py
%load_ext autoreload
%autoreload 2

s3 = S3()

In [ ]:
if not exists(BERT_VOCAB):
    s3.download(S3_BERT_VOCAB, BERT_VOCAB)

In [ ]:
torch.set_grad_enabled(False)

In [30]:
words_vocab = BERTVocab.load(BERT_VOCAB)
tags_vocab = BIOTagsVocab(TAGS)

config = RuBERTConfig()
emb = BERTEmbedding.from_config(config)
encoder = BERTEncoder.from_config(config)
ner = BERTNERHead(config.emb_dim, len(tags_vocab))
model = BERTNER(emb, encoder, ner)
model.eval()

model.emb.load(BERT_EMB) # default downloaded embeddings
model.encoder.load(MODEL_ENCODER) # custom pretrained model encoder
model.head.load(MODEL_NER) # custom pretrained model head
model = model.to(DEVICE)

encoder = BERTInferEncoder(
    words_vocab,
    seq_len=SEQ_LEN, batch_size=BATCH_SIZE
)

decoder = BERTTagDecoder(tags_vocab)
infer = BERTNERInfer(model, encoder, decoder)

In [ ]:
# inference

default = ['Европейский союз добавил в санкционный список девять политических деятелей из самопровозглашенных республик Донбасса — Донецкой народной республики (ДНР) и Луганской народной республики (ЛНР) — в связи с прошедшими там выборами. Об этом говорится в документе, опубликованном в официальном журнале Евросоюза. В новом списке фигурирует Леонид Пасечник, который по итогам выборов стал главой ЛНР. Помимо него там присутствуют Владимир Бидевка и Денис Мирошниченко, председатели законодательных органов ДНР и ЛНР, а также Ольга Позднякова и Елена Кравченко, председатели ЦИК обеих республик. Выборы прошли в непризнанных республиках Донбасса 11 ноября. На них удержали лидерство действующие руководители и партии — Денис Пушилин и «Донецкая республика» в ДНР и Леонид Пасечник с движением «Мир Луганщине» в ЛНР. Президент Франции Эмманюэль Макрон и канцлер ФРГ Ангела Меркель после встречи с украинским лидером Петром Порошенко осудили проведение выборов, заявив, что они нелегитимны и «подрывают территориальную целостность и суверенитет Украины». Позже к осуждению присоединились США с обещаниями новых санкций для России.',]
custom = ['Ваш произвольный текст']

chunk = custom if CUSTOM_TUNING else default

markups = list(infer(chunk)) # chunk is about 1000 chars? - https://github.com/natasha/naeval/blob/52c4a508bf212b95d4e610cfe1b5e23b8ca94d2f/naeval/ner/models/slovnet.py#L16

spans = sum(len(_.spans) for _ in markups)

data = [_.as_json for _ in markups]

print(markups)

from ipymarkup import show_span_ascii_markup as show_markup
show_markup(markups[0].text, markups[0].spans)